In [10]:
from bigml.api import BigML
api = BigML(project='project/5d94a426eba31d46690001cd')

Sources and datasets creation.

In [2]:
train_full_source = api.create_source('./datasets/train_full_df.csv')
api.ok(train_full_source) # waits the OK response from BigML server

train_full_dataset = api.create_dataset(train_full_source)
api.ok(train_full_dataset)
%store train_full_dataset # stores the python object for other notebooks

Stored 'train_full_dataset' (dict)
Stored 'train_training_dataset' (dict)


We split the *train_full_dataset* into a **80/20 random split**.

In [3]:
train_training_dataset = api.create_dataset(train_full_dataset, {"name": "train | training", "sample_rate": 0.8, "seed": "myseed"})
                                                                # "seed" is the hash string used for randomisation
api.ok(train_training_dataset)
%store train_training_dataset

train_validation_dataset = api.create_dataset(
    train_full_dataset, {"name": "train | validation",
                        "sample_rate": 0.8, "out_of_bag": True, "seed": "myseed"})
                         # "out_of_bag" takes the remaining 20% data from the previous split, "seed" matching the previous hash string
api.ok(train_validation_dataset)
%store train_validation_dataset

Stored 'train_validation_dataset' (dict)


We create the Kaggle test source and dataset.

In [4]:
kaggle_test_source = api.create_source('./datasets/test_df.csv')
api.ok(kaggle_test_source)

kaggle_test_dataset = api.create_dataset(kaggle_test_source)
api.ok(kaggle_test_dataset)
%store kaggle_test_dataset

Stored 'kaggle_test_dataset' (dict)


We split our train_training dataset into 10 ones.

In [19]:
import csv
with open('datasets_id.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Name", "dataset_id"])

def train_training_split(rate):
    dataset = api.create_dataset(train_training_dataset, 
                                 {"name": f"train | training | {rate}", "sample_rate": rate, "seed": "myseed"})
    api.ok(dataset)
    with open('splitted_datasets.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([f"train | training | {rate}", dataset['resource']])   

In [20]:
train_training_dataset = api.get_dataset("dataset/5db2c2ebe47684746800a438")
for n in range(1, 10):
    train_training_split(n/10)